### Sistema p-Fuzzy Contínuo: Predador-Presa

#### 📘 Introdução

Este notebook implementa um **sistema dinâmico p-fuzzy contínuo** para modelar a interação entre **presas e predadores**, baseado no exemplo do livro:

> Barros, L. C., Bassanezi, R. C., & Lodwick, W. A. (2017).  
> *A First Course in Fuzzy Logic, Fuzzy Dynamical Systems, and Biomathematics*

---

#### 🎯 O que é um Sistema p-Fuzzy Contínuo?

Um **sistema p-fuzzy contínuo** modela a evolução temporal usando **equações diferenciais ordinárias (EDOs)** onde a função de evolução é determinada por **regras fuzzy**.

#### Sistema Contínuo:
$$\frac{dx}{dt} = f(x)$$

Onde $f(x)$ é definido por um **Sistema de Inferência Fuzzy (FIS)**.

---

#### 🦊 Modelo Lotka-Volterra Fuzzy

O modelo clássico de Lotka-Volterra descreve:

$$\frac{dx_1}{dt} = \alpha x_1 - \beta x_1 x_2 \quad \text{(presas)}$$

$$\frac{dx_2}{dt} = \delta x_1 x_2 - \gamma x_2 \quad \text{(predadores)}$$

##### Versão Fuzzy:
As taxas de mudança são determinadas por regras fuzzy ao invés de parâmetros fixos $\alpha, \beta, \gamma, \delta$.

**Vantagens:**
- ✅ Incorpora conhecimento especialista
- ✅ Não requer calibração paramétrica precisa
- ✅ Captura comportamentos não-lineares complexos

---

## Instalação e Imports

In [ ]:
# Instalar pyfuzzy-toolbox
!pip install pyfuzzy-toolbox matplotlib numpy -q

In [ ]:
import sys
import os

import fuzzy_systems as fs
from fuzzy_systems.dynamics import PFuzzyContinuous
from fuzzy_systems.inference import MamdaniSystem
import numpy as np
import matplotlib.pyplot as plt

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# Criar sistema Mamdani para predador-presa

# fis = MamdaniSystem.load('prey_predator.json')

fis = MamdaniSystem(name="Predador-Presa Continuo")

# ========================================
# ENTRADAS: Presas (x) e Predadores (y)
# ========================================

# Universo de discurso: [0, 100] para ambas
fis.add_input('prey', (0, 100))
fis.add_input('predator', (0, 100))

# Termos linguísticos (4 níveis como no livro)
for var in ['prey', 'predator']:
    fis.add_term(var, 'B', 'gaussian', (0, 12))    # Baixa
    fis.add_term(var, 'MB', 'gaussian', (100/3, 12))   # Média-Baixa
    fis.add_term(var, 'MA', 'gaussian', (2*100/3, 12))   # Média-Alta
    fis.add_term(var, 'A', 'gaussian', (100, 12))    # Alta

fis.plot_variables()

In [ ]:
# ========================================
# SAÍDAS: Variação de Presas e Predadores
# ========================================

# Saída 1: Δprey (variação de presas)
var_prey_universe = (-2,2)
lrg = 0.5

fis.add_output('var_prey', var_prey_universe)
fis.add_term('var_prey', 'B_n', 'triangular', (-lrg,0,0))
fis.add_term('var_prey', 'MB_n', 'triangular', (-2*lrg,-lrg, 0))
fis.add_term('var_prey', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_prey', 'A_n', 'trapezoidal', (-4*lrg,-4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_prey', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_prey', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_prey', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_prey', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg,4*lrg))

var_predator_universe = (-2,2)
lrg = 0.5

fis.add_output('var_predator', var_predator_universe)
fis.add_term('var_predator', 'B_n', 'triangular', (-lrg,0,0))
fis.add_term('var_predator', 'MB_n', 'triangular', (-2*lrg,-lrg, 0))
fis.add_term('var_predator', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_predator', 'A_n', 'trapezoidal', (-4*lrg,-4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_predator', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_predator', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_predator', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_predator', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg,4*lrg))


fis.plot_variables(['var_prey','var_predator'])


In [ ]:
# ========================================
# BASE DE REGRAS (16 regras)
# ========================================

# Definir regras usando sintaxe de tupla plana
# Formato: ('prey_term', 'predator_term', 'd_prey_term', 'd_predator_term')

rules = [
    # Linha 1: prey = B (baixa)
    ('B', 'B', 'MB_p', 'MB_n'),
    ('B', 'MB', 'B_p', 'MB_n'),
    ('B', 'MA', 'B_n', 'MA_n'),
    ('B', 'A', 'MB_n', 'A_n'),
    
    # Linha 2: prey = MB (média-baixa)
    ('MB', 'B', 'MA_p', 'B_n'),
    ('MB', 'MB', 'MB_p', 'B_n'),
    ('MB', 'MA', 'B_n', 'MB_n'),
    ('MB', 'A', 'MB_n', 'MA_n'),
    
    # Linha 3: prey = MA (média-alta)
    ('MA', 'B', 'MB_p', 'MA_p'),
    ('MA', 'MB', 'B_p', 'MB_p'),
    ('MA', 'MA', 'MB_n', 'B_p'),
    ('MA', 'A', 'MA_n', 'B_p'),
    
    # Linha 4: prey = A (alta)
    ('A', 'B', 'B_n', 'A_p'),
    ('A', 'MB', 'MB_n', 'MA_p'),
    ('A', 'MA', 'MA_n', 'MB_p'),
    ('A', 'A', 'A_n', 'B_p')
]

# Adicionar regras ao FIS usando a nova sintaxe
fis.add_rules(rules)
fis.plot_rule_matrix()

# Use fis.export_rules(filename) to save the rules
# Use fis.import_rules(filename) to import rules
# Use fis.save(filename) to save the fis interily
# Use fis.load(filename) to load a fis

In [ ]:
# Criar sistema p-fuzzy contínuo
pfuzzy = PFuzzyContinuous(
    fis=fis,
    mode='absolute',  # dx/dt = f(x)
    state_vars=['prey', 'predator'],
    method='euler'  # Runge-Kutta 4ª ordem (mais preciso)
)



In [ ]:
# Condições iniciais
x0 = {'prey': 40, 'predator': 20}

# Intervalo de tempo
t_span = (0, 250)

# Simular
t_adapt, x_adapt = pfuzzy.simulate(
    x0={'prey': 40, 'predator': 20},
    t_span=(0, 250),
    adaptive=True,
    tolerance=1e-4,
)




In [ ]:
# Plot da dinâmica temporal
fig, ax = pfuzzy.plot_trajectory(
    variables=['prey', 'predator'],
    figsize=(14, 6),
    title='Sistema Lotka-Volterra Fuzzy Contínuo',
    xlabel='Tempo',
    ylabel='População'
)

plt.show()

In [ ]:
# Plot do espaço de fase
fig, ax = pfuzzy.plot_phase_space(
    'prey', 'predator',
    figsize=(8, 5),
    title='Espaço de Fase: Ciclo Predador-Presa'
)

plt.show()

In [ ]:
# Criar grade para campo vetorial
prey_grid = np.linspace(10, 90, 12)
pred_grid = np.linspace(10, 90, 12)
P, D = np.meshgrid(prey_grid, pred_grid)

# Calcular vetores de campo (dx/dt, dy/dt)
dP = np.zeros_like(P)
dD = np.zeros_like(D)


for i in range(len(prey_grid)):
    for j in range(len(pred_grid)):
        state = {'prey': P[j, i], 'predator': D[j, i]}
        result = fis.evaluate(state)
        dP[j, i] = result['var_prey']
        dD[j, i] = result['var_predator']

# Plotar
fig, ax = plt.subplots(figsize=(12, 10))

# Campo vetorial
ax.quiver(P, D, dP, dD, alpha=0.5, color='gray', width=0.003)

# Trajetória
ax.plot(trajectory[:, 0], trajectory[:, 1], 'b-', linewidth=2.5, 
        label='Trajetória', alpha=0.8)
ax.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=12, 
        label='Inicial', zorder=5)
ax.plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=12, 
        label='Final', zorder=5)

ax.set_xlabel('Presas', fontsize=13)
ax.set_ylabel('Predadores', fontsize=13)
ax.set_title('Campo Vetorial + Trajetória do Sistema p-Fuzzy', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()


In [ ]:
# Diferentes condições iniciais
initial_conditions = [
    {'prey': 30, 'predator': 15},
    {'prey': 50, 'predator': 25},
    {'prey': 70, 'predator': 35},
    {'prey': 40, 'predator': 40}
]

colors = ['blue', 'red', 'green', 'purple']

# Criar figura com 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

print("🔄 Simulando múltiplas condições iniciais...\n")

for i, x0 in enumerate(initial_conditions):
    print(f"{i+1}. Presas={x0['prey']}, Predadores={x0['predator']}")
    
    # Simular
    traj = pfuzzy.simulate(x0=x0, t_span=t_span)
    time = pfuzzy.time
    
    # Plot temporal
    ax1.plot(time, traj[:, 0], '--', color=colors[i], alpha=0.6, linewidth=1.5)
    ax1.plot(time, traj[:, 1], '-', color=colors[i], linewidth=2, 
             label=f"IC{i+1}: ({x0['prey']}, {x0['predator']})")
    
    # Plot espaço de fase
    ax2.plot(traj[:, 0], traj[:, 1], color=colors[i], linewidth=2.5, alpha=0.7)
    ax2.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=10)

# Configurar plot temporal
ax1.set_xlabel('Tempo', fontsize=12)
ax1.set_ylabel('População', fontsize=12)
ax1.set_title('Dinâmica Temporal\n(linha sólida=predador, tracejada=presa)', 
              fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Configurar plot espaço de fase
ax2.set_xlabel('Presas', fontsize=12)
ax2.set_ylabel('Predadores', fontsize=12)
ax2.set_title('Espaço de Fase\n(círculo=condição inicial)', 
              fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 100)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()


In [ ]:
# Exportar para CSV
pfuzzy.to_csv('/tmp/predator_prey_continuous.csv')

print("✅ Dados exportados para: /tmp/predator_prey_continuous.csv")
print("\n📄 Primeiras 5 linhas:")

import pandas as pd
df = pd.read_csv('/tmp/predator_prey_continuous.csv')
print(df.head(10))